# Analysis for project 1

Run PCA, MDS, t-SNE, and UMAP on TCGA gene expression data. Inspect variation of metadata features across reduced dimensions.

In [ ]:
import numpy as np
import pandas as pd
from anndata import AnnData
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

# import embedding functions
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
from umap import UMAP

## Read and clean the data

In [ ]:
# read expression data
expr = pd.read_csv('data/TCGA.HNSC.expression.txt', sep='\t', header=0, index_col=[0,1])
print("expression data shape: ", expr.shape)
expr.head()[0:5]

In [ ]:
# read meta data
meta = pd.read_csv('data/TCGA.HNSC.metadata.txt', sep='\t', header=0, index_col=0)
print("meta data shape: ", meta.shape)
meta.head()

In [ ]:
# get meta data per sample
meta = expr.reset_index()[['patient_id','sample_id']].join(meta, on='patient_id').set_index(['patient_id','sample_id'])
print("meta data shape: ", meta.shape)
meta.head()

We have 545 samples from 500 patients. 20531 features (genes) were quantified from each sample

## Inspect mean-variance relationship of features

In [ ]:
mean_var = pd.DataFrame()
mean_var['mean'] = expr.mean(axis=0)
mean_var['var'] = expr.var(axis=0)
f = sns.scatterplot(data=mean_var, x='mean', y='var', s = 5, alpha=0.5)

In [ ]:
# log transform axes to see more clearly
f = sns.scatterplot(data=mean_var, x='mean', y='var', s = 5, alpha=0.5)
f.set(xscale="log", yscale="log")

Data is heteroscedastic. We will need to stabilize the variance of the data before proceeding.

In [ ]:
# log1p transform expression data (can't use log because of zeros)
expr_norm = expr.apply(lambda x: np.log1p(x), axis=0)
mean_var['mean'] = expr_norm.mean(axis=0)
mean_var['var'] = expr_norm.var(axis=0)
sns.scatterplot(data=mean_var, x='mean', y='var', s = 5, alpha=0.5)

## Normalize, scale, center, and embed the data

In [ ]:
# put into Annotated Data object for nice organization
adata = AnnData(expr, obs=meta, var=expr.columns.to_frame(name='gene'), dtype=np.float32)
adata.obs_names = expr.index.get_level_values(1)
adata.var_names = expr.columns

# remove zeros
adata = adata[:, adata.X.sum(axis=0) > 0]

# log(x+1) transform each value in matrix
adata.X_normed = np.log1p(adata.X)

# scale only, don't center (PCA centers internally)
adata.X_scaled = scale(adata.X_normed, with_mean=False, axis = 1)

# scale and center
adata.X_standardized = scale(adata.X_normed, with_mean=True, axis=1)

In [ ]:
# setup 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(10,10))
for i, method in enumerate(['pca', 'mds', 'tsne', 'umap']):
	if method == 'pca':
		# get embedding
		embedding = PCA().fit_transform(adata.X_scaled)
	elif method == 'mds':
		# get embedding
		embedding = MDS(normalized_stress='auto').fit_transform(adata.X_standardized)
	elif method == 'tsne':
		# get embedding
		embedding = TSNE().fit_transform(adata.X_standardized)
	elif method == 'umap':
		# get embedding
		embedding = UMAP().fit_transform(adata.X_standardized)

	# get embedding
	method = method.upper()
	
	# plot embedding
	sns.scatterplot(x=embedding[:,0], y=embedding[:,1], s=5, alpha=0.5, hue=adata.obs['histological_grade'], ax=axs[int(i/2), i%2])
	axs[int(i/2), i%2].set_title(method)
	axs[int(i/2), i%2].set(xlabel=method+'_1', ylabel=method+'_2')

	# hide legend
	axs[int(i/2), i%2].legend_.remove()

In [ ]:
adata._sanitize() # Transform string annotations to categoricals.
adata.write_h5ad('data/TCGA.HNSC.embedded.h5ad')